In [1]:
# get only the longest protein sequence for each protein-coding gene
import os
import sys
import subprocess

In [12]:
gff_fname = "/home/varabyou/genomicData/hg38/annotations/chess2.2.gff"
prot_tmp_fname = "/home/varabyou/genomicData/hg38/annotations/chess2.2.prot.tmp.fasta"
prot_longest_fname = "/home/varabyou/genomicData/hg38/annotations/chess2.2.protein.longest.fa"
prot_all_fname = "/home/varabyou/genomicData/hg38/annotations/chess2.2.protein.all.fa"
ref_fname = "/home/varabyou/genomicData/hg38/hg38_p8.fa"

gffread_path = "/home/varabyou/soft/gffread/gffread"

In [3]:
subprocess.call([gffread_path,gff_fname,"-g",ref_fname,"-y",prot_tmp_fname,])

0

In [13]:
# the dictionary is going to hold the following
# 1st level - geneID
# 2nd level - sequence
# 3rd level - all transcript ids that contain this sequence
res = dict()

cur_seq = ""

gID = ""
tID = ""

first = True

with open(prot_tmp_fname,"r") as inFP:
    for line in inFP.readlines():
        if line[0]==">":
            if not first:
                res.setdefault(gID,dict()).setdefault(cur_seq,[]).append(tID)
            first = False
            cur_seq = ""
            tID = line.strip()[1:]
            gID = ".".join(tID.split(".")[:2])
        else:
            cur_seq+=line.strip()
    if not first:
        res.setdefault(gID,dict()).setdefault(cur_seq,[]).append(tID)

In [14]:
# output longest sequences only
outFP = open(prot_longest_fname,"w+")
for k,v in res.items():
    max_len = max(map(len,list(v)))
    longests = [x for x in list(v) if len(x) == max_len]
    for l in longests:
        outFP.write(">"+"|".join(v[l])+"\n")
        outFP.write(l+"\n")
outFP.close()

In [15]:
# output all non-redundant sequences only
outFP = open(prot_all_fname,"w+")
for k,v in res.items():
    for seq in (list(v)):
        outFP.write(">"+"|".join(v[seq])+"\n")
        outFP.write(seq+"\n")
outFP.close()